## Дипломная работа на тему "Предсказание цен на поддержанные автомобили в США в соответствии с данными сайта craigslist.org"

## 1. Описание

Craigslist.org - одна из крупнейших интернет-площадок электронных объявлений в мире, особенно пользующаяся большой популярностью в США. Подавляющую часть этого сайта занимают объявления о продаже поддержанных автомобилей. 

Представленный [набор данных](https://www.kaggle.com/austinreese/craigslist-carstrucks-data) обновляется каждый месяц и содержит исчерпывающую информацию об автомобилях, которую предоставляет сайт craigslist.org, включая сведения о марке и модели автомобиля, его состоянии, цене продажи, пробеге и т.д. Актуальная версия набора данных представлена от 09.2020

В рамках выполнения дипломной работы необходимо решить задачу регрессии и построить соответствующую модель, предсказывающую цены на поддержанные автомобили. Для проверки качества регрессии использовать метрику RMSE.

## 2. Data Understanding

### 2.1. Подключение библиотек

In [30]:
# modelling helpers
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

# data handling and preparation
import numpy as np
import pandas as pd

# visualization
from jupyterthemes import jtplot
from scipy.stats import norm
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
jtplot.style()

### 2.2. Вспомогательные функции

In [43]:
def print_obj_columns(data):
    obj_cols = list(data.select_dtypes(include=['object']).columns)
    for col in data[obj_cols]:
        print(f'{col}:')
        print(f'Len of unique items list: {len(data[col].unique())}')
        if len(data[col].unique()) > 20:
            print(f'{data[col].unique()[:5]}')
        else:
            print(f'{data[col].unique()}')
        print('-'*40)

        
def plot_variable_importance(X, y):
    tree = RandomForestRegressor(random_state = 419)
    tree.fit(X, y)
    plot_model_var_imp(tree, X, y)
    
    
def plot_model_var_imp(model, X, y):
    imp = pd.DataFrame( 
        model.feature_importances_  , 
        columns = ['Importance'] , 
        index = X.columns 
    )
    imp = imp.sort_values(['Importance'], ascending=True)
    imp[:10].plot(kind = 'barh')

### 2.3. Загрузка данных

In [38]:
data = pd.read_csv('./data/external/vehicles.csv')
data.head()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,drive,size,type,paint_color,image_url,description,county,state,lat,long
0,7184791621,https://duluth.craigslist.org/ctd/d/duluth-200...,duluth / superior,https://duluth.craigslist.org,6995,2000.0,gmc,new sierra 1500,excellent,8 cylinders,...,4wd,NaN,NaN,red,https://images.craigslist.org/00n0n_f06ykBMcdh...,2000 *** GMC New Sierra 1500 Ext Cab 157.5 WB...,NaN,mn,46.8433,-92.2550
1,7184773187,https://duluth.craigslist.org/cto/d/saginaw-20...,duluth / superior,https://duluth.craigslist.org,8750,2013.0,hyundai,sonata,excellent,4 cylinders,...,fwd,NaN,NaN,grey,https://images.craigslist.org/00d0d_kgZ6xoeRw2...,For Sale: 2013 Hyundai Sonata GLS - $8750. O...,NaN,mn,46.9074,-92.4638
2,7193375964,https://newhaven.craigslist.org/cto/d/stratfor...,new haven,https://newhaven.craigslist.org,10900,2013.0,toyota,prius,good,4 cylinders,...,fwd,NaN,NaN,blue,https://images.craigslist.org/00d0d_3sHGxPbY2O...,2013 Prius V Model Two. One owner—must sell my...,NaN,ct,41.1770,-73.1336
3,7195108810,https://albuquerque.craigslist.org/cto/d/albuq...,albuquerque,https://albuquerque.craigslist.org,12500,2003.0,mitsubishi,lancer,good,4 cylinders,...,4wd,mid-size,sedan,grey,https://images.craigslist.org/00m0m_4a8Pb6JbMG...,"2003 Mitsubishi Lancer Evolution, silver. Abo...",NaN,nm,35.1868,-106.6650
4,7184712241,https://duluth.craigslist.org/ctd/d/rush-city-...,duluth / superior,https://duluth.craigslist.org,16995,2007.0,gmc,sierra classic 2500hd,good,8 cylinders,...,4wd,full-size,truck,white,https://images.craigslist.org/01414_g093aPtSMW...,"**Bad Credit, No Credit... No Problem!**2007 G...",NaN,mn,45.6836,-92.9648


In [34]:
data.shape

(423857, 25)

**Описание переменных**

Признаки:
* идентификатор
* url
* регион
* url региона
* год выпуска
* производитель
* модель
* состояние автомобиля
* количество цилиндров
* тип топлива
* пробег
* тех.паспорт
* тип коробки передач
* VIN-номер
* привод
* размер
* тип кузова
* цвет
* url изображения
* описание
* округ
* штат
* координаты широты
* координаты долготы

Целевая переменная:
* цена (в долларах США)

### 2.4. EDA

#### 2.4.1. Предварительный анализ

Проведем предварительный анализ данных для идентификации выбросов, пропусков и признаков, которые являются лишними и от которых можно избавиться.

In [45]:
data.describe()

,id,price,year,odometer,county,lat,long
count,4.238570e+05,4.238570e+05,328743.000000,2.705850e+05,0.0,324404.000000,324404.000000
mean,7.190602e+09,5.779437e+04,2009.827358,9.942578e+04,NaN,38.731429,-93.949367
std,5.159269e+06,1.002581e+07,10.135809,1.204833e+05,NaN,5.902977,17.930566
min,7.178302e+09,0.000000e+00,1900.000000,0.000000e+00,NaN,-80.386400,-161.394000
25%,7.186688e+09,4.499000e+03,2007.000000,4.432400e+04,NaN,35.054500,-106.678000
50%,7.192068e+09,9.995000e+03,2012.000000,9.174900e+04,NaN,39.366900,-88.009900
75%,7.194909e+09,1.899500e+04,2016.000000,1.372370e+05,NaN,42.517075,-80.832500
max,7.197006e+09,3.808256e+09,2021.000000,1.000000e+07,NaN,80.383400,115.524000


In [36]:
print_obj_columns(data)

url:
Len of unique items list: 423857
['https://duluth.craigslist.org/ctd/d/duluth-2000-gmc-new-sierra-1500-ext-cab/7184791621.html'
 'https://duluth.craigslist.org/cto/d/saginaw-2013-hyundai-sonata-gls/7184773187.html'
 'https://newhaven.craigslist.org/cto/d/stratford-2013-toyota-prius-2-wagon-4d/7193375964.html'
 'https://albuquerque.craigslist.org/cto/d/albuquerque-mitsubishi-lancer-evolution/7195108810.html'
 'https://duluth.craigslist.org/ctd/d/rush-city-2007-gmc-sierra-classic/7184712241.html']
----------------------------------------
region:
Len of unique items list: 404
['duluth / superior' 'new haven' 'albuquerque' 'lubbock'
 'fargo / moorhead']
----------------------------------------
region_url:
Len of unique items list: 413
['https://duluth.craigslist.org' 'https://newhaven.craigslist.org'
 'https://albuquerque.craigslist.org' 'https://lubbock.craigslist.org'
 'https://fargo.craigslist.org']
----------------------------------------
manufacturer:
Len of unique items list: 44

In [37]:
data.isna().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year             95114
manufacturer    110615
model            98473
condition       247138
cylinders       226178
fuel             96643
odometer        153272
title_status     96098
transmission     95792
vin             239437
drive           192738
size            321230
type            182700
paint_color     201654
image_url        94196
description      94203
county          423857
state                0
lat              99453
long             99453
dtype: int64

**Результаты предварительного анализа**

1. Необходимо избавиться от признаков, которые в дальнейшем не повлияют (или влияние будет крайне мало) на конечную стоимость продаваемого автомобиля (идентификаторы, url'ы, VIN-номер, описание, координаты и т.д.).
2. Заменить нулевые показания стоимости и пробега на средние по рынку.
3. Заменить значения года производства на взвешенные.
4. Заменить пропуски на средние или медианные значения (в зависимости от признака).
5. Закодировать категориальные признаки с использованием OneHotEncoding (для возможности построения линейных моделей и получения корректных результатов).
6. Проведение over/undersampling нецелесообразно. У нас имеется достаточная выборка данных для построения модели.


#### 2.4.2. Подготовка данных

Избавимся от лишних признаков

In [22]:
data.columns

Index(['id', 'url', 'region', 'region_url', 'price', 'year', 'manufacturer',
       'model', 'condition', 'cylinders', 'fuel', 'odometer', 'title_status',
       'transmission', 'vin', 'drive', 'size', 'type', 'paint_color',
       'image_url', 'description', 'county', 'state', 'lat', 'long'],
      dtype='object')

In [46]:
drops = ['id', 'url', 'region', 'region_url', 'county', 'lat', 'long', 'state', 'image_url', 'description', 'vin', 'model']
reduced = data.drop(columns=drops)
reduced.head()

,price,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color
0,6995,2000.0,gmc,excellent,8 cylinders,gas,167783.0,clean,automatic,4wd,NaN,NaN,red
1,8750,2013.0,hyundai,excellent,4 cylinders,gas,90821.0,clean,automatic,fwd,NaN,NaN,grey
2,10900,2013.0,toyota,good,4 cylinders,hybrid,92800.0,clean,automatic,fwd,NaN,NaN,blue
3,12500,2003.0,mitsubishi,good,4 cylinders,gas,NaN,clean,manual,4wd,mid-size,sedan,grey
4,16995,2007.0,gmc,good,8 cylinders,diesel,254217.0,clean,automatic,4wd,full-size,truck,white


...